In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV,LeaveOneOut,cross_val_score
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,make_scorer,matthews_corrcoef
from sklearn.metrics import confusion_matrix,roc_curve,auc
import matplotlib.pyplot as plt
import matplotlib
import copy
from pathlib import Path
import pathlib as path
import joblib
import shap
import os

In [ ]:
# 数据读取
columns_backward=-1
train_path= r"Q:\ALK_1_Wild\3.ALK_300nM_RANDOM_ECFP4\ECFP4_filter387_RF\ChEMBL_TargetID_CHEMBL4247_1810ECFP4_filter387-RF-111_tr_1212.csv"
test_path = r"Q:\ALK_1_Wild\3.ALK_300nM_RANDOM_ECFP4\ECFP4_filter387_RF\ChEMBL_TargetID_CHEMBL4247_1810ECFP4_filter387-RF-111_te_598.csv"
P_tr=path.WindowsPath(train_path)
P_te=path.WindowsPath(test_path)
train_data=pd.read_csv(train_path)
test_data=pd.read_csv(test_path)
# 截取读取
train_x_index = train_data.columns[:columns_backward]
train_x=train_data[train_x_index]
train_y=train_data['label']
test_x_index = test_data.columns[:columns_backward]
test_x=test_data[test_x_index]
test_y=test_data['label']

In [ ]:
# 模型训练：记得改参数
rf=RandomForestClassifier(random_state=3,max_features=40,max_leaf_nodes=110,n_estimators =40).fit(train_x,train_y)

# 测试集-全局解释

In [ ]:
# 创建SHAP解释器
explainer = shap.TreeExplainer(rf,data=train_x)

# 计算 SHAP 值列表
shap_values = explainer.shap_values(test_x) # 计算 SHAP 值：shap_values是一个包含SHAP值的列表，每个元素对应于一个类的SHAP值（对于二分类问题，这个列表通常有两个元素）
shap_values1= shap_values[1]                # 提取 SHAP 值：提取了列表中第二个元素（索引1），即对应于类1的SHAP值。shap_values1是一个二维数组，每行对应一个样本，每列对应一个特征，这个数组表示每个样本的每个特征对预测为类1的贡献
shap_values1qu= shap_values[:,:,1]          # 曲丹尝试

# 计算 SHAP 值矩阵
shap_values2= explainer(test_x)             # 计算 SHAP 值：shap_values2是一个包含SHAP值的Explanation对象，是一个三维数组，其中第一维是样本索引，第二维是特征索引，第三维是类索引
shap_values3= shap_values2[:,:,1]           # 提取 SHAP 值：从三维数组中提取出对应于类1的SHAP值。shap_values2[:,:,1]是一个二维数组，其中每行对应一个样本，每列对应一个特征，表示每个样本的每个特征对预测为类1的贡献

shap.plots.bar(shap_values2[:,:,1] ) # shap.plots.bar 需要Explanation对象格式传入

features=train_x_index

In [ ]:
# 调试 SHAP 值维度不匹配问题
print(f"Shape of test_x: {test_x.shape}")
print(f"Shape of shap_values: {shap_values.shape}") 
print(f"Shape of shap_values1: {shap_values1.shape}")
print(f"Shape of shap_values1qu: {shap_values1qu.shape}")
print(f"Shape of shap_values2: {shap_values2.shape}")
print(f"Shape of shap_values3: {shap_values3.shape}")

In [ ]:
def save_shap_values(shap_values, output_path):
    # 确保输出目录存在
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 保存SHAP值到指定路径的文件中
    np.savetxt(output_path, shap_values[1], delimiter=",")  # shap_values[1]的含义待定？我觉着如果按这样算得出的CSV提取的数据不对，待考证

# 示例调用
output_path = r"Q:\ALK_1_Wild\7.ALK_300nM_SHAP\Model_7D\Model_7D_SHAP_values_test.csv"
save_shap_values(shap_values, output_path)

In [ ]:
shap.plots.bar(shap_values2[:,:,1], show_data=True) # shap.plots.bar 需要Explanation对象格式传入

In [ ]:
# shap.summary_plot(shap_values[1], feature_names=features, plot_type="bar")      # shap_values1提取公式会影响结果

# shap.summary_plot(shap_values[:,:,1], feature_names=features, plot_type="bar")  # shap_values1提取公式会影响结果，与shap_values3结果相同

shap.summary_plot(shap_values2[:,:,1], feature_names=features, plot_type="bar") # shap_values3与shap_values1都可作图

In [ ]:
# shap.summary_plot(shap_values[1], test_x)       # shap_values1提取公式会影响结果，Error：The shape of the shap_values matrix does not match the shape of the provided data matrix.

# shap.summary_plot(shap_values[:,:,1], test_x)   # shap_values1提取公式会影响结果，与shap_values3结果相同

shap.summary_plot(shap_values2[:,:,1], test_x)  # shap_values3与shap_values1都可作图

In [ ]:
# shap.plots.heatmap(shap_values[1][:4000])       # shap_values1不可作图，Error: 'numpy.ndarray' object has no attribute 'values'

# shap.plots.heatmap(shap_values[:,:,1][:4000])   # shap_values1不可作图，Error: 'numpy.ndarray' object has no attribute 'values'

shap.plots.heatmap(shap_values2[:,:,1][:4000])  # shap_values3可作图

In [ ]:
# # 单个样本解释
# shap.waterfall_plot(shap_values2[:,:,1][400,:,1])
# features=train_x_index

# 训练集-全局解释

In [ ]:
# 训练集-全局解释

# 计算 SHAP 值列表
shap_values4 = explainer.shap_values(train_x) # 计算 SHAP 值：shap_values是一个包含SHAP值的列表，每个元素对应于一个类的SHAP值（对于二分类问题，这个列表通常有两个元素）
shap_values5= shap_values4[1]                 # 提取 SHAP 值：提取了列表中第二个元素（索引1），即对应于类1的SHAP值。shap_values1是一个二维数组，每行对应一个样本，每列对应一个特征，这个数组表示每个样本的每个特征对预测为类1的贡献
shap_values5qu= shap_values4[:,:,1]           # 曲丹尝试

# 计算 SHAP 值矩阵
shap_values6= explainer(train_x)              # 计算 SHAP 值：shap_values2是一个包含SHAP值的Explanation对象，是一个三维数组，其中第一维是样本索引，第二维是特征索引，第三维是类索引
shap_values7= shap_values6[:,:,1]             # 提取 SHAP 值：从三维数组中提取出对应于类1的SHAP值。shap_values2[:,:,1]是一个二维数组，其中每行对应一个样本，每列对应一个特征，表示每个样本的每个特征对预测为类1的贡献

shap.plots.bar(shap_values6[:,:,1] ) # shap.plots.bar 需要Explanation对象格式传入

features=train_x_index

In [ ]:
# 调试 SHAP 值维度不匹配问题
print(f"Shape of train_x: {train_x.shape}")
print(f"Shape of shap_values4: {shap_values4.shape}") 
print(f"Shape of shap_values5: {shap_values5.shape}")
print(f"Shape of shap_values5qu: {shap_values5qu.shape}")
print(f"Shape of shap_values6: {shap_values6.shape}")
print(f"Shape of shap_values7: {shap_values7.shape}")

In [ ]:
def save_shap_values(shap_values, output_path):
    # 确保输出目录存在
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 保存 SHAP 值到指定路径的文件中
    np.savetxt(output_path, shap_values4[1], delimiter=",")

# 示例调用
output_path = r"Q:\ALK_1_Wild\7.ALK_300nM_SHAP\Model_7D\Model_7D_SHAP_values_train.csv"
save_shap_values(shap_values, output_path)

In [ ]:
shap.plots.bar(shap_values6[:,:,1], show_data=True) # shap.plots.bar 需要Explanation对象格式传入

In [ ]:
# shap.summary_plot(shap_values4[1], feature_names=features, plot_type="bar")      # shap_values1提取公式会影响结果

# shap.summary_plot(shap_values4[:,:,1], feature_names=features, plot_type="bar")  # shap_values1提取公式会影响结果，与shap_values3结果相同

shap.summary_plot(shap_values6[:,:,1], feature_names=features, plot_type="bar") # shap_values3与shap_values1都可作图

In [ ]:
# shap.summary_plot(shap_values4[1], train_x)       # shap_values1提取公式会影响结果，Error：The shape of the shap_values matrix does not match the shape of the provided data matrix.

# shap.summary_plot(shap_values4[:,:,1], train_x)   # shap_values1提取公式会影响结果，与shap_values3结果相同

shap.summary_plot(shap_values6[:,:,1], train_x)  # shap_values3与shap_values1都可作图

In [ ]:
# shap.plots.heatmap(shap_values4[1][:4000])       # shap_values1不可作图，Error: 'numpy.ndarray' object has no attribute 'values'

# shap.plots.heatmap(shap_values4[:,:,1][:4000])   # shap_values1不可作图，Error: 'numpy.ndarray' object has no attribute 'values'

shap.plots.heatmap(shap_values6[:,:,1][:4000])  # shap_values3可作图